### Group: SAXA 4
### Bassma Ali · Andrew Singh · Andy Oliver · Destiny Floyd-McGuiness . Vahid Dabbaghi Sadr

Combined Architecture Diagram

```text
               ┌────────────────────┐
               │  final_data        │
               │ (with text_clean   │
               │  + 17_impact_type) │
               └────────┬───────────┘
                        │
                        ▼
              [Train / Val / Test Split]
                        │
                        ▼
        ┌──────────────────────────────────┐
        │  TF-IDF Vectorizer (text_clean)  │
        └───────────────┬──────────────────┘
                        │
                        ▼
           Classical ML Classifier
          (LogReg / Linear SVM + optional
           structured features)
                        │
           y_pred_ml, p_ml, metrics
                        │
        ┌───────────────┴───────────────────┐
        │                                   │
        ▼                                   ▼
 Low-confidence / audit subset        All (or subset)
 sent to LLM with:                    sent to LLM for:
 - text_clean                         - explanations
 - key metadata                       - policy language
 - y_pred_ml + p_ml (optional)        
        │                                   │
        ▼                                   ▼
  LLM label + rationale          Natural language rationales
        │                                   │
        └───────────────┬───────────────────┘
                        ▼
          Final outputs (labels + explanations
          + performance comparison ML vs LLM)


In [ ]:
# ---------------------------
# 📦 1. Import Required Packages
# ---------------------------

# Core data handling
import numpy as np
import pandas as pd

# Train/val/test split
from sklearn.model_selection import train_test_split

# Text feature extraction (TF-IDF, CountVectorizer, etc.)
# We import TF-IDF now because we know we will use it later,
# but we are NOT selecting a classifier yet.
from sklearn.feature_extraction.text import TfidfVectorizer

# Visualization (optional for EDA checks or confusion matrices later)
import matplotlib.pyplot as plt
import seaborn as sns

# NOTE:
# We intentionally do NOT import any models yet (e.g., LogisticRegression, SVM)
# because the model selection will be decided in a later step.
# ---------------------------


In [ ]:
# ---------------------------
# 📥 2. Load preprocessed dataset produced in Notebook 1
# ---------------------------

df = pd.read_csv("final_data_preprocessed.csv")

print("Dataset loaded with shape:", df.shape)
df.head()


Dataset loaded with shape: (1232, 45)


,2_use_case_name,3_agency,3_abr,4_bureau,8_topic_area,10_commercial_ai,11_purpose_benefits,12_outputs,16_dev_stage,17_impact_type,...,55_independent_eval,56_monitor_postdeploy,57_autonomous_impact,62_disparity_mitigation,63_stakeholder_consult,65_appeal_process,17_impact_type_clean,row_completeness,text_combined_raw,text_clean
0,Non-Intrusive Inspection (NII) 3D Imaging Tool,Department of Homeland Security,DHS,CBP,Law & Justice,None of the above.,"Utilizes AI/ML to generate high resolution, ra...",Detection alerts for Items of Interest.,Implementation and Assessment,Neither,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Neither,0.492063,"Utilizes AI/ML to generate high resolution, ra...","utilizes ai/ml to generate high resolution, ra..."
1,Babel,Department of Homeland Security,DHS,CBP,Law & Justice,None of the above.,CBP uses this tool to conduct targeted queries...,Babel utilizes AI modules for Text detection a...,Implementation and Assessment,Rights-Impacting\n,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Rights-Impacting,0.444444,CBP uses this tool to conduct targeted queries...,cbp uses this tool to conduct targeted queries...
2,Fivecast ONYX,Department of Homeland Security,DHS,CBP,Law & Justice,None of the above.,The system enhances CBP's capability to monito...,The system aggregates vast amounts of publicl...,Implementation and Assessment,Rights-Impacting\n,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Rights-Impacting,0.412698,The system enhances CBP's capability to monito...,the system enhances cbp's capability to monito...
3,Advanced Analytics for X-ray Images (AAXI),Department of Homeland Security,DHS,CBP,Law & Justice,None of the above.,AAXI aims to address the problem of anomaly de...,AAXI produces bounding boxes around an anomaly...,Acquisition and/or Development,Rights-Impacting\n,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Rights-Impacting,0.365079,AAXI aims to address the problem of anomaly de...,aaxi aims to address the problem of anomaly de...
4,Advanced Trade Analytics Platform (ATAP),Department of Homeland Security,DHS,CBP,Diplomacy & Trade,None of the above.,To create efficiencies and unlock key insights...,Model output is provided in dashboards and oth...,Operation and Maintenance,Neither,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Neither,0.523810,To create efficiencies and unlock key insights...,to create efficiencies and unlock key insights...
